In [ ]:
%pylab inline 
import pandas as pd 
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq

NNNNNNNTGACTaaccaattcagtcgactggatccggtaccaaggtcgggcaggaaNNNNNNNNNgagggcctatttcccatgattccttcatatttgcatatacgatacaaggctgttagagagataattagaattaatttgactgtaaacacaaagatattagtacaaaatacgtgacgtagaaagtaataatttcttgggtagtttgcagttttaaaattatgttttaaaatggactatcatatgcttaccgtaacttgaaagtatttcgatttcttggctttatatatcttgtggaaaggacgaaacaccGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAANNNNNNNNNNNNAAAAAGTGGCACCGAGTCGGTGCgcattacagtacgaagatgcatagctttttttaagcttgggccgctcgaggtacctctctaTGACTNNNNNNN

UMI - Vector -- 9nt Barcode - Promoter - Guide - Spacer - 12nt Barcode  - linker and terminator - vector - UMI 

Vector1: aaccaattcagtcgactggatccggtaccaaggtcgggcaggaa

9nt Barcode:

U6: gagggcctatttcccatgattccttcatatttgcatatacgatacaaggctgttagagagataattagaattaatttgactgtaaacacaaagatattagtacaaaatacgtgacgtagaaagtaataatttcttgggtagtttgcagttttaaaattatgttttaaaatggactatcatatgcttaccgtaacttgaaagtatttcgatttcttggctttatatatcttgtggaaaggacgaaacacc

Guide: 
ggcccagactgagcacgtga

spacer: GTTTAAGAGCTAAGCTGGAA

linker-terminator-vector: AAAAAGTGGCACCGAGTCGGTGCgcattacagtacgaagatgcatagctttttttaagcttgggccgctcgaggtacctctctaTGACT

In [18]:
read1 = "TAATTGATGACTAACCAATTCAGTCGACTGGATCCGGTACCAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTTGTGGAAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAACCAACAAACTTTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA"


In [19]:
read2="TGACTAACCAATTCAGTCGACTGGATCCGGTACCAACTTGAAAGTATTTCGATTTCTTGGCTTTATATATCTTGTGGAAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAACCAACAAACTTTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCAGGCTCCC"


In [55]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def merge_and_trim_sequences(seq1, seq2, umi1, umi2):
    # Align the two sequences using local alignment
    alignments = pairwise2.align.localms(seq1, seq2, 2, -1, -3, -0.5, one_alignment_only=True)
    aligned_seq1, aligned_seq2, score, start, end = alignments[0]

    # Initialize the consensus sequence with the first aligned sequence
    consensus = list(aligned_seq1)

    # Merge the aligned sequences into the consensus sequence
    for i, (base1, base2) in enumerate(zip(aligned_seq1, aligned_seq2)):
        if base1 == '-':
            consensus[i] = base2
        elif base2 == '-':
            continue
        elif base1 == 'N' and base2 != 'N':
            consensus[i] = base2
        elif base2 == 'N' and base1 != 'N':
            consensus[i] = base1
        elif base1 != base2:
            # Choose the base from seq2 if a mismatch occurs, unless seq2 contains 'N'
            consensus[i] = base2 if base2 != 'N' else base1

    # Join consensus list into string and remove gaps
    consensus = ''.join(consensus).replace('-', '')

    # Find UMI1 and remove everything to the left
    umi1_index = consensus.find(umi1)
    if umi1_index != -1:
        consensus = consensus[umi1_index + len(umi1):]

    # Find UMI2 and remove everything to the right (reverse complement)
    def reverse_complement(seq):
        complement = str.maketrans("ATGC", "TACG")
        return seq.translate(complement)[::-1]

    umi2_rc = reverse_complement(umi2)
    umi2_index = consensus.find(umi2_rc)
    if umi2_index != -1:
        consensus = consensus[:umi2_index]

    return consensus



In [ ]:
#It looks liek barcode 1 starts an index late 

In [94]:
seq1='CGACACTCTTTCCCAACACGACGCTCTTCCGATCGGAGGGGGTGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGCAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA'

In [95]:
seq2 = 'TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCATATAACA'

In [96]:

umi1 = 'GAGGGGG'
umi2 = 'TGTTATA'
consensus_sequence = merge_and_trim_sequences(seq1, seq2,umi1, umi2 )

print("Consensus Sequence:", consensus_sequence)

Consensus Sequence: TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA


Consensus Sequence: TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA

TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAG

Barcode1: GACGAAACA

U6: CC
guide - GGCCCAGACTGAGCACGTGA
spacer - GTTTAAGAGCTAAGCTGGAA

Barcode2: ACTTTCCTCAGT


AAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA

In [117]:
alignments = pairwise2.align.localms(consensus_sequence, 
                                     pre_barcode_vector1, 2, -1, -3, -0.5, one_alignment_only=True)
aligned_vector_seq, aligned_consensus_seq, score, start, end = alignments[0]
aligned_vector_seq, aligned_consensus_seq, score, start, end

('TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAG-AAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA',
 '-----------------------------GTACCAAGGTCGGGCAGGAA---------------------------------------------------------------------------------------------------------------------------------------------------------',
 35.0,
 29,
 49)

'TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA'

In [118]:
consensus_sequence[start:end]

'GTACCAAGGTCGGGCAGAAG'

In [114]:
consensus[end:]

'ACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA'

In [98]:
#consensus_sequence
pre_barcode_vector1 = 'GTACCAAGGTCGGGCAGGAA' #9nt after 
pre_barcode_vector2 = 'GTTTAAGAGCTAAGCTGGAA' #12nt after 

barcode1=extract_barcode(consensus_sequence, pre_barcode_vector1, barcode_length=9)
barcode2=extract_barcode(consensus_sequence, pre_barcode_vector2,  barcode_length=12)

In [106]:
barcode1=extract_barcode(consensus_sequence, pre_barcode_vector1, barcode_length=9)
barcode1

'GACGAAACA'

In [100]:
barcode1, barcode2

('GACGAAACA', 'ACTTTCCTCAGT')

In [102]:
consensus_sequence

'TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA'

In [132]:
vectorq = 'TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAG'
vector1 = 'aaccaattcagtcgactggatccggtaccaaggtcgggcaggaa'.upper()

alignments = pairwise2.align.localms(consensus_seq , pre_barcode_vector1, 2, -1, -3, -0.5, one_alignment_only=True)

# Display the best alignment
for alignment in alignments:
    print(format_alignment(*alignment))

30 GTACCAAGGTCGGGCAG-AA
   ||||||||||||||||| ||
 1 GTACCAAGGTCGGGCAGGAA
  Score=35



In [153]:
from Bio import pairwise2

def extract_barcode(consensus, vector, barcode_length=12):
    
    alignments = pairwise2.align.localms(consensus, vector, 2, -1, -3, -0.5, one_alignment_only=True)
    if alignments:
        aligned_consensus_seq, aligned_vector_seq, score, start, end = alignments[0]
        # Calculate the exact index in the consensus without considering gaps
        gapless_end = start + sum(1 for base in aligned_vector_seq if base != '-')
        barcode_start = gapless_end
        barcode_end = barcode_start + barcode_length
        return consensus[barcode_start:barcode_end]
    return ""


In [155]:
consensus_sequence, pre_barcode_vector1, barcode_length=9

TypeError: cannot unpack non-iterable int object

In [184]:
barcode_length = 9 
consensus = consensus_sequence
vector = pre_barcode_vector1 

alignments = pairwise2.align.localms(consensus, vector, 2, -1, -3, -0.5, one_alignment_only=True)

aligned_consensus_seq, aligned_vector_seq, score, start, end = alignments[0]
# Calculate the exact index in the consensus without considering gaps
gapless_end = start + sum(1 for base in aligned_vector_seq if base != '-')
barcode_start = gapless_end
barcode_end = barcode_start + barcode_length
consensus[barcode_start:barcode_end]

<ipython-input-184-e71997b5d28c>:9: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gapless_end = start + sum(1 for base in aligned_vector_seq if base != '-')


'GACGAAACA'

In [166]:
gapless_end = start + sum(1 for base in aligned_vector_seq if base != '-')
gapless_end
barcode_start = gapless_end

<ipython-input-166-07ed7a2868f6>:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gapless_end = start + sum(1 for base in aligned_vector_seq if base != '-')


In [92]:

consensus_sequence = "TGACTAACCAATTCAGTCGACTGGATCCGGTACCAAGGTCGGGCAGAAGGACGAAACACCGGCCCAGACTGAGCACGTGAGTTTAAGAGCTAAGCTGGAAACTTTCCTCAGTAAAAAGTGGCACCGAGTCGGTGCGCATTACAGTACGAAGATGCATAGCTTTTTTTAAGCTTGGGCCGCTCGAGGTACCTCTCTAAGTCA"
vector2 = 'GTTTAAGAGCTAAGCTGGAA'
vector1 = 'GTACCAAGGTCGGGCAGGAA'


# Perform local alignment explicitly with a scoring matrix
alignments = pairwise2.align.localms(consensus_sequence , vector1, 2, -1, -3, -0.5, one_alignment_only=True)

# Display the best alignment
for alignment in alignments:
    print(format_alignment(*alignment))

27 CCGGTACCAAGGTCGGGCAG-AA
   |||||||||||||||||||| ||
 1 CCGGTACCAAGGTCGGGCAGGAA
  Score=41



In [91]:
"ccggtaccaaggtcgggcaggaa".upper()

'CCGGTACCAAGGTCGGGCAGGAA'

In [84]:
def extract_barcode(consensus, vector, barcode_length=12):
    # Align the consensus sequence to the known vector sequence
    alignments = pairwise2.align.localms(consensus, vector, 2, -1, -3, -0.5, one_alignment_only=True)
    if alignments:
        aligned_vector_seq, aligned_consensus_seq, score, start, end = alignments[0]
        barcode_start = end
        barcode_end = barcode_start + barcode_length
        barcode = consensus[barcode_start:barcode_end]
        return barcode
    return ""


In [86]:
barcode2 = extract_barcode(consensus, vector, barcode_length=12)

In [88]:
len(barcode2), barcode2

(12, 'ACTTTCCTCAGT')

In [89]:
vector1 = "ccggtaccaaggtcgggcaggaa"
barcode1 = extract_barcode(consensus, vector, barcode_length=9)

In [90]:
len(barcode1), barcode1

(9, 'ACTTTCCTC')

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import robin_stocks as rs

with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_ticker = {executor.submit(fetch_stock_price, ticker): ticker for ticker in Tickers}

    for future in as_completed(future_to_ticker):
        ticker, stock_price = future.result()
        handle_trades(ticker, stock_price, PC, PD, CC, CD, current_risk, limit)


put_options = rs.options.find_options_by_expiration(ticker, '2024-05-17', 'put')

In [185]:
import concurrent.futures
import time

# Function to simulate data fetching
def fetch_data(task_id):
    print(f"Starting task {task_id}")
    time.sleep(task_id)  # Simulate time delay of fetching data
    return f"Task {task_id} completed"

# Main function to run parallel tasks
def run_parallel_tasks():
    tasks = [1, 2, 3, 4, 5]  # Simulate different tasks with varying times
    start_time = time.time()

    # Using ThreadPoolExecutor to run tasks in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Map each task to the executor
        results = list(executor.map(fetch_data, tasks))

    # Output results
    for result in results:
        print(result)

    # Calculate total time taken
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")

if __name__ == "__main__":
    run_parallel_tasks()


Starting task 1Starting task 2

Starting task 3
Starting task 4
Starting task 5
Task 1 completed
Task 2 completed
Task 3 completed
Task 4 completed
Task 5 completed
Time taken: 5.066293954849243 seconds
